In [1]:
import time
import argparse
import math
import os
import torch
import torch.nn as nn
from torch import optim
import numpy
import matplotlib
from matplotlib import pyplot as plt


# Import your model files.
from model import make_model, Classifier, NoamOpt, LabelSmoothing, fgim_attack
from data import prepare_data, non_pair_data_loader, get_cuda, pad_batch_seuqences, load_human_answer,\
    id2text_sentence, to_var, calc_bleu


In [2]:
from attributedict.collections import AttributeDict

In [9]:
args = AttributeDict({'id_pad':0, 'id_unk':1, "id_bos" : 2, 'id_eos':3,'task':'dialect',\
                     'word_to_id_file': '', 'data_path' :'../../data/uae-eg/processed_files/', \
                     'word_dict_max_num' :2, 'batch_size' :128, 'max_sequence_length' :64, 'num_layers_AE':2,
                     'transformer_model_size':256, 'transformer_ff_size' : 1024,'latent_size': 256, \
                     'word_dropout': 1.0, 'embedding_dropout':0.5,'learning_rate':0.001, 'label_size':1})

In [10]:
args.id_bos

2

In [11]:
args.if_load_from_checkpoint = True
args.checkpoint_name = "1588135513"

In [12]:
def add_log(ss):
    now_time = time.strftime("[%Y-%m-%d %H:%M:%S]: ", time.localtime())
    print(now_time + ss)
    with open(args.log_file, 'a') as f:
        f.write(now_time + str(ss) + '\n')
    return


def add_output(ss):
    with open(args.output_file, 'a') as f:
        f.write(str(ss) + '\n')
    return


def preparation():
    # set model save path
    if args.if_load_from_checkpoint:
        timestamp = args.checkpoint_name
    else:
        timestamp = str(int(time.time()))
        print("create new model save path: %s" % timestamp)
    args.current_save_path = 'save/%s/' % timestamp
    args.log_file = args.current_save_path + time.strftime("log_%Y_%m_%d_%H_%M_%S.txt", time.localtime())
    args.output_file = args.current_save_path + time.strftime("output_%Y_%m_%d_%H_%M_%S.txt", time.localtime())
    print("create log file at path: %s" % args.log_file)

    if os.path.exists(args.current_save_path):
        add_log("Load checkpoint model from Path: %s" % args.current_save_path)
    else:
        os.makedirs(args.current_save_path)
        add_log("Path: %s is created" % args.current_save_path)

    # set task type
    if args.task == 'yelp':
        args.data_path = '../../data/yelp/processed_files/'
    elif args.task == 'amazon':
        args.data_path = '../../data/amazon/processed_files/'
    elif args.task == 'dialect':
        args.data_path = '../../data/uae-eg/processed_files/'
    elif args.task == 'imagecaption':
        pass
    else:
        raise TypeError('Wrong task type!')

    # prepare data
    args.id_to_word, args.vocab_size, \
    args.train_file_list, args.train_label_list = prepare_data(
        data_path=args.data_path, max_num=args.word_dict_max_num, task_type=args.task
    )
    return

In [13]:
preparation()

ae_model = get_cuda(make_model(d_vocab=args.vocab_size,
                               N=args.num_layers_AE,
                               d_model=args.transformer_model_size,
                               latent_size=args.latent_size,
                               d_ff=args.transformer_ff_size,
))
dis_model = get_cuda(Classifier(latent_size=args.latent_size, output_size=args.label_size))

create log file at path: save/1588135513/log_2020_04_29_23_34_15.txt
[2020-04-29 23:34:15]: Load checkpoint model from Path: save/1588135513/
prepare data ...
Load word-dict with 94055 size and 2 max_num.


In [14]:
ae_model.load_state_dict(torch.load(args.current_save_path + 'ae_model_params.pkl'))
dis_model.load_state_dict(torch.load(args.current_save_path + 'dis_model_params.pkl'))

<All keys matched successfully>

In [24]:
def eval_iters(ae_model, dis_model):
    eval_data_loader = non_pair_data_loader(
        batch_size=1, id_bos=args.id_bos,
        id_eos=args.id_eos, id_unk=args.id_unk,
        max_sequence_length=args.max_sequence_length, vocab_size=args.vocab_size
    )
    eval_file_list = [
        args.data_path + 'dialect_dev.uae',
        args.data_path + 'dialect_dev.eg',
    ]
    eval_label_list = [
        [0],
        [1],
    ]
    ind2label = {0.0:'UAE',1.0:"EGY"}
    eval_data_loader.create_batches(eval_file_list, eval_label_list, if_shuffle=False)
#     gold_ans = load_human_answer(args.data_path)
#     assert len(gold_ans) == eval_data_loader.num_batch

    count = 0
    add_log("Start eval process.")
    ae_model.eval()
    dis_model.eval()
    for it in range(eval_data_loader.num_batch):
        batch_sentences, tensor_labels, \
        tensor_src, tensor_src_mask, tensor_tgt, tensor_tgt_y, \
        tensor_tgt_mask, tensor_ntokens = eval_data_loader.next_batch()

        print("\n\n------------%d------------" % it)
        print("Origin_input")
        print(id2text_sentence(tensor_tgt_y[0], args.id_to_word))
        print("Origin label is ", ind2label[tensor_labels.cpu().item()])

        latent, out = ae_model.forward(tensor_src, tensor_tgt, tensor_src_mask, tensor_tgt_mask)
        generator_text = ae_model.greedy_decode(latent,
                                                max_len=args.max_sequence_length,
                                                start_id=args.id_bos)
#         print("------------------------")
#         print("autoencoder output:")
#         print(id2text_sentence(generator_text[0], args.id_to_word))

        # Define target label
        target = get_cuda(torch.tensor([[1.0]], dtype=torch.float))
        if tensor_labels[0].item() > 0.5:
            target = get_cuda(torch.tensor([[0.0]], dtype=torch.float))
        print("Target_label:", ind2label[target.cpu().item()])

        modify_text = fgim_attack(dis_model, latent, target, ae_model, args.max_sequence_length, args.id_bos,
                                        id2text_sentence, args.id_to_word, '')
        add_output(modify_text)
        count += 1
        if count >= 20:
            break
    return

In [25]:
eval_iters(ae_model, dis_model)

Load data from ../../data/uae-eg/processed_files/dialect_dev.uae ../../data/uae-eg/processed_files/dialect_dev.eg !
Create 59225 batches with 1 batch_size
[2020-04-29 23:56:56]: Start eval process.


------------0------------
Origin_input
user الانضباط تغرم hash num الف درهم وتوجه له <UNK> <UNK> القمصان <UNK> في مباراه راس الخيمه
Origin label is  UAE
Target_label: EGY
epsilon: 7.0
| It  1 | dis model pred 0.0490 |
user hash الشعبيه في num الف جنيه <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> له دعوه صحيح المقصود راس في <UNK>
epsilon: 6.3
| It  2 | dis model pred 0.9093 |
user hash الشعبيه في num الف جنيه <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> له دعوه ادفع مستوي برده <UNK> في جذب
epsilon: 5.67
| It  3 | dis model pred 0.9643 |
user hash الشعبيه في num الف جنيه <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> له دعوه ادفع مستوي برده <UNK> في جذب
epsilon: 5.103
| It  4 | dis model pred 0.9733 |
user hash الشعبيه في num الف جنيه <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> ده

| It  4 | dis model pred 0.7002 |
اقم الصلاه اذا يعقلون فيها سبيل الي الفجر وكان وعيون كان لك الذين امنوا وجنه ٤٢ سوره الاسراء hash
epsilon: 5.248800000000001
| It  5 | dis model pred 0.7474 |
اقم الصلاه اذا يعقلون فيها سبيل الي الفجر وكان وعيون كان لك الذين امنوا وجنه ٤٢ سوره الاسراء hash


------------5------------
Origin_input
اجمل النساء امراه سهرت و تعبت و ربت و اعطت دون مقابل num تدعي امي
Origin label is  UAE
Target_label: EGY
epsilon: 7.0
| It  1 | dis model pred 0.2667 |
نفسي اجمل امراه لن نجد من <UNK> و <UNK> و <UNK> num اه دون عدم num فلا تعتقد انها الدنيا بتضحك
epsilon: 6.3
| It  2 | dis model pred 0.6664 |
نفسي اجمل النساء تجمع و من <UNK> <UNK> num و <UNK> num دون عدم num تقف ليك ولا
epsilon: 5.67
| It  3 | dis model pred 0.8512 |
نفسي اجمل امراه لن نجد من <UNK> و <UNK> num و <UNK> اه دون عدم تعتقد منك تحبك التي الدنيا num
epsilon: 5.103
| It  4 | dis model pred 0.8848 |
نفسي اجمل امراه لن نجد من <UNK> و <UNK> num و <UNK> اه دون عدم تعتقد منك تحبك التي الدنيا num
epsilon: 4

Target_label: EGY
epsilon: 7.0
| It  1 | dis model pred 0.1811 |
user عملت الطيب اوي و فرح لم وليه <UNK> حاجه و انت نايم
epsilon: 6.3
| It  2 | dis model pred 0.9349 |
user عملت الطيب اوي و فرح لم وليه <UNK> حاجه و انت نايم
epsilon: 5.67
| It  3 | dis model pred 0.9472 |
user عملت طوله محمد و فرح هما اللي جيت <UNK> كويس لا يوجد
epsilon: 5.103
| It  4 | dis model pred 0.9549 |
user عملت طوله محمد و فرح هما اللي جيت <UNK> كويس لا يوجد
epsilon: 4.5927
| It  5 | dis model pred 0.9600 |
user عملت طوله محمد و فرح هما اللي جيت <UNK> كويس لا يوجد
epsilon: 8.0
| It  1 | dis model pred 0.1811 |
user عملت الطيب اوي و قصه واحد <UNK> جيت علشان اعلم و انت يستمع user
epsilon: 7.2
| It  2 | dis model pred 0.9494 |
user عملت الطيب اوي و قصه واحد <UNK> جيت علشان اعلم و انت يستمع دي
epsilon: 6.48
| It  3 | dis model pred 0.9575 |
user عملت الطيب اوي و قصه واحد <UNK> جيت علشان اعلم و انت يستمع دي
epsilon: 5.832000000000001
| It  4 | dis model pred 0.9630 |
user عملت الطيب اوي و حصل واحد <UNK> علشان قدرت س

| It  2 | dis model pred 0.9896 |
user ستيشن مش اول وقت <UNK> بص <UNK> <UNK> num <UNK> مش حاجه num وكل شخص
epsilon: 5.67
| It  3 | dis model pred 0.9903 |
user ستيشن مش اول وقت <UNK> بص <UNK> <UNK> num <UNK> مش حاجه num وكل شخص
epsilon: 5.103
| It  4 | dis model pred 0.9909 |
user ستيشن مش اول وقت <UNK> بص <UNK> <UNK> num <UNK> مش حاجه num وكل شخص
epsilon: 4.5927
| It  5 | dis model pred 0.9913 |
user ستيشن مش اول وقت <UNK> بص <UNK> <UNK> num <UNK> مش حاجه num وكل شخص
epsilon: 8.0
| It  1 | dis model pred 0.3259 |
user ستيشن مش اول وقت <UNK> <UNK> بص num <UNK> <UNK> مش حاجه num وكل شخص
epsilon: 7.2
| It  2 | dis model pred 0.9938 |
user ستيشن مش اول وقت <UNK> <UNK> بص num <UNK> <UNK> مش حاجه num وكل شخص
epsilon: 6.48
| It  3 | dis model pred 0.9941 |
user ستيشن مش اول وقت <UNK> <UNK> بص num <UNK> <UNK> مش حاجه num وكل شخص
epsilon: 5.832000000000001
| It  4 | dis model pred 0.9944 |
user ستيشن مش اول وقت <UNK> <UNK> بص num <UNK> <UNK> مش حاجه num وكل شخص
epsilon: 5.248800000000001
| It 

In [26]:
print(ae_model)

EncoderDecoder(
  (encoder): Encoder(
    (layers): ModuleList(
      (0): EncoderLayer(
        (self_attn): MultiHeadedAttention(
          (linears): ModuleList(
            (0): Linear(in_features=256, out_features=256, bias=True)
            (1): Linear(in_features=256, out_features=256, bias=True)
            (2): Linear(in_features=256, out_features=256, bias=True)
            (3): Linear(in_features=256, out_features=256, bias=True)
          )
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (feed_forward): PositionwiseFeedForward(
          (w_1): Linear(in_features=256, out_features=1024, bias=True)
          (w_2): Linear(in_features=1024, out_features=256, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (sublayer): ModuleList(
          (0): SublayerConnection(
            (norm): LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): SublayerConnection(
            (norm): LayerNorm()

In [1]:
def load_word_dict_info(word_dict_file, max_num):
    id_to_word = []
    with open(word_dict_file, 'r') as f:
        lines = f.readlines()
        for line in lines:
            item = line.strip()
            item_list = item.split('\t')
            word = item_list[0]
            if len(item_list) > 1:
                num = int(item_list[1])
                if num < max_num:
                    break
            id_to_word.append(word)
    print("Load word-dict with %d size and %d max_num." % (len(id_to_word), max_num))
    return id_to_word, len(id_to_word)

In [3]:
x1, x2 = load_word_dict_info("../../data/uae-eg/processed_files/word_to_id.txt", 2)

Load word-dict with 94055 size and 2 max_num.
